In [26]:
import json
import pandas as pd
import socket
import numpy as np


"""
Basic code to establish connection to the Thermostat using TCP protocol. 

Eunji (2025-10-08)
"""



def send_scpi_command(host, port, command):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((host, port))
        sock.sendall(command.encode()) ### Send a command
        response = sock.recv(1024).decode() ### recieve the return
        return response.strip()

ip = '192.168.1.26' ### Found with arp /a and router  - reserved
port = 23  ### Specified on github thermostat page

device_id = send_scpi_command(ip, port, '*IDN?\n')
print("Device ID:", device_id)



### send a single command:
command = 'report\n'
result = send_scpi_command(ip, port, command)
print("Return:", result)

# result is the JSON string you printed
data = json.loads(result)                       # -> list of dicts for ch0, ch1
df = pd.DataFrame(data).set_index("channel").T  # rows=fields, cols=channel

# (optional) choose just the fields you care about, in a nice order
fields = [
    "temperature","adc","sens","i_tec","i_set","pid_output",
    "dac_value","dac_feedback","tec_i","tec_u_meas",
    "pid_engaged","current_swapped","interval","time"
]
df = df.reindex(fields)

# (optional) nicer float formatting
pd.options.display.float_format = "{:,.6g}".format

display(df)   # in Jupyter; or print(df) in a terminal

Device ID: { "error": "invalid input" }
Return: [{"channel":0,"time":5.0556073000000004e5,"interval":1.2e-1,"adc":null,"sens":null,"temperature":null,"pid_engaged":true,"current_swapped":true,"i_set":-0e0,"dac_value":1.5000171662031794e0,"dac_feedback":1.5e0,"i_tec":1.5150000000000001e0,"tec_i":-5.249999999999977e-2,"tec_u_meas":-3.599999999999959e-2,"pid_output":0e0},{"channel":1,"time":5.0556079000000004e5,"interval":1.2e-1,"adc":7.079710217501409e-1,"sens":7.665692445329528e3,"temperature":3.200010705793301e1,"pid_engaged":true,"current_swapped":true,"i_set":2.7017394744544276e-1,"dac_value":1.5524309077389238e0,"dac_feedback":1.549e0,"i_tec":1.549e0,"tec_i":-1.3499999999999956e-1,"tec_u_meas":3.360000000000003e-1,"pid_output":2.7017394744544276e-1}]


channel,0,1
temperature,NaN,32.0001
adc,NaN,0.707971
sens,NaN,"7,665.69"
i_tec,1.515,1.549
i_set,-0,0.270174
pid_output,0,0.270174
dac_value,1.50002,1.55243
dac_feedback,1.5,1.549
tec_i,-0.0525,-0.135
tec_u_meas,-0.036,0.336
